参考文档：
https://langchain-ai.github.io/langgraph/agents/multi-agent/

如果单个代理需要专注于多个领域或管理许多工具，则它可能会遇到困难。要解决此问题，您可以将代理分解为更小的独立代理，并将它们组合成一个多代理系统。

在多代理系统中，代理之间需要相互通信。它们通过切换来实现这一点，切换是一种原语，用于描述将控制权移交给哪个代理以及要发送给该代理的有效负载。

两种最流行的多代理架构是：

Supervisor — 各个代理由中央 Supervisor 代理协调。主管控制所有通信流和任务委派，根据当前上下文和任务要求决定调用哪个代理。
Swarm — 代理根据其专长动态地将控制权移交给彼此。系统会记住最后一个处于活动状态的座席，确保在后续交互中，与该座席的对话恢复。




使用 langgraph-supervisor 库创建 supervisor 多智能体系统：


pip install langgraph-supervisor
或者
uv pip install langgraph-supervisor -i https://pypi.tuna.tsinghua.edu.cn/simple/


In [ ]:

# 先创建llm
import os
from langchain.chat_models import init_chat_model

API_KEY = "sk-123"

BASE_URL = "https://api.deepseek.com"

os.environ["OPENAI_API_KEY"] = API_KEY
os.environ["OPENAI_API_BASE"] = BASE_URL


llm = init_chat_model("openai:deepseek-chat")


from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

def book_hotel(hotel_name: str):
    """Book a hotel"""
    return f"Successfully booked a stay at {hotel_name}."

def book_flight(from_airport: str, to_airport: str):
    """Book a flight"""
    return f"Successfully booked a flight from {from_airport} to {to_airport}."

flight_assistant = create_react_agent(
    model=llm,
    tools=[book_flight],
    prompt="You are a flight booking assistant",
    name="flight_assistant"
)

hotel_assistant = create_react_agent(
    model=llm,
    tools=[book_hotel],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)

supervisor = create_supervisor(
    agents=[flight_assistant, hotel_assistant],
    model=llm,
    prompt=(
        "You manage a hotel booking assistant and a"
        "flight booking assistant. Assign work to them."
    )
).compile()

for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "book a flight from BOS to JFK and a stay at McKittrick Hotel ，请用中文回答"
            }
        ]
    }
):
    print(chunk)
    print("\n")

{'supervisor': None}


{'hotel_assistant': {'messages': [AIMessage(content='我已经为您转接到酒店预订助手，可以为您预订麦基特里克酒店（McKittrick Hotel）。请问您需要预订的具体日期和入住天数是多少？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 156, 'total_tokens': 190, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 64}, 'prompt_cache_hit_tokens': 64, 'prompt_cache_miss_tokens': 92}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': '23d984a3-00a7-46e5-bfc5-94289331b13c', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, name='hotel_assistant', id='run--0664506c-8c57-4c73-8e90-1dbdda32ca14-0', usage_metadata={'input_tokens': 156, 'output_tokens': 34, 'total_tokens': 190, 'input_token_details': {'cache_read': 64}, 'output_token_details': {}}), AIMessage(content='Transferring back to supervisor', additional_kwargs={}, response_metadata={'__is_handoff_back': True}, na